In [2]:
# HACK: use project root as the working directory 
from pathlib import Path

while Path.cwd().name != 'language-model-toxicity':
    %cd ..

/home/sam/Desktop/research/language-model-toxicity


In [3]:
import pandas as pd
import torch
from transformers import GPT2Tokenizer

from models.affect_lm import AffectGPT2LMHeadModel
from utils.constants import OUTPUT_DIR

In [ ]:
model_dir = OUTPUT_DIR / 'affect' / 'affect_100k_1layer_randomsample_frozen' / 'finetune_output'
model = AffectGPT2LMHeadModel.from_pretrained(model_dir)

In [4]:
affect2vocab = model.affect.affect2vocab
weight = affect2vocab.weight
bias = affect2vocab.bias

In [23]:
values, indices = weight.max(dim=0)

In [12]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [13]:
vocab = tokenizer.get_vocab()
vocab, _ = zip(*vocab.items())
vocab = pd.Series(vocab)
vocab

0                    !
1                    "
2                    #
3                    $
4                    %
             ...      
50252         Ġregress
50253        ĠCollider
50254      Ġinformants
50255           Ġgazed
50256    <|endoftext|>
Length: 50257, dtype: object

In [24]:
vocab[indices.tolist()]

31030         Ġbastard
247                  Ļ
50256    <|endoftext|>
25                   :
247                  Ļ
50256    <|endoftext|>
50256    <|endoftext|>
50256    <|endoftext|>
dtype: object